# GoPro Photos to Crop KMZ

<a target="_blank" href="https://colab.research.google.com/github/nasaharvest/street2sat/blob/main/notebooks/GoPro2CropKMZ.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Author**: Ivan Zvonkov

**Last Modified**: Jan 20, 2025

**Description**: Converts GoPro photos to crop type points. Specifically the notebook:

1. Downloads GoPro photos from Google Cloud or Google Drive.
2. Create a dataframe from photos.
3. Extract dates and coordinates.
4. Classify as crop or not crop.
5. Segment crops in crop photos.
6. Filter out low confidence predictions.
7. Move coordinate to field
8. Get Admin Zones for each point
9. Create KMZ file



## Important Prerequisite

Before running any cell in the notebook,
1. Click the drop down triangle on the top right hand side and select "Change Runtime Type".
2. Click the T4 radio button under Hardware Accelerator and click save.

This will allow the CropSegmentation model to run much faster.

In [ ]:
# Required packages
!pip install exifread utm simplekml -q

## 1. Download GoPro photos

In [ ]:
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("Select Location of GoPro Photos")
print("1. Google Cloud Storage (Preferred)")
print("2. Google Drive")
selection = input("Enter selection [ 1 / 2 ]: ")

#########################################
# Download GoPros from Google Cloud
#########################################
GCLOUD_PATH_INPUT = ""
if selection == "1":

    # Login to Google Cloud, once you run this cell click on the space after "browser:" to enter the code
    !gcloud auth login
    # Your current project will be [None] and that is okay.

    print("\nCopy and paste the path of the Google Cloud Storage folder with GoPro Photos")
    print("(Example: street2sat-uploaded/KENYA_v2/2021_07_13_T2/100GOPRO)")
    GCLOUD_PATH_INPUT = input("Path: ")

    GCLOUD_PATH = f"gs://{GCLOUD_PATH_INPUT}/*"
    PREFIX = GCLOUD_PATH_INPUT.replace("street2sat-uploaded/", "").replace("/", "_")
    Path(PREFIX).mkdir(exist_ok=True)
    print(f"\nReady to download images from \n{GCLOUD_PATH} to {PREFIX}")

    # Check amount of photos
    !gsutil du $GCLOUD_PATH | wc -l

    confirm = input("Confirm? [y/n]: ")
    if confirm == "y":
        # 20 mins for 10k images
        !gsutil -m cp -r $GCLOUD_PATH $PREFIX
    else:
        raise print("Cancelled")

#########################################
# Download GoPros from Google Drive
#########################################
elif selection == "2":
    print("\nPath of Google Drive Folder with photos:")
    print("(Example: /content/drive/MyDrive/2021-07-05-T1)")
    PREFIX = input("").replace("/content/", "")

else:
    raise ValueError(f"Invalid selection: '{selection}'")


In [ ]:
country_is_right_hand_drive = {
    "KENYA": False,
    "MADAGASCAR": True
}

# If country not in prefix, "is_right_hand_drive" has to be set manually
is_right_hand_drive = None

for country, is_rhs in country_is_right_hand_drive.items():
    if country in PREFIX.upper():
        is_right_hand_drive = is_rhs
        break


assert is_right_hand_drive != None, "Drive direction not derived, set 'is_right_hand_drive' manually"
print("Assuming " + ("right hand drive" if is_right_hand_drive else "left hand drive for:"))
print(PREFIX)


## 2. Create dataframe from available photos

In [ ]:
from datetime import datetime
from shapely.geometry import Point
from tqdm import tqdm

import exifread
import geopandas as gpd
import pandas as pd

tqdm.pandas()

image_folder = Path(f"/content/{PREFIX}")
if (not image_folder.exists()):
    print("STOP: Update image_folder to match the folder of images you downloaded")
else:
    gopro_photo_paths = list(image_folder.glob("**/*.JPG")) # May need to switch to .jpg
    df = pd.DataFrame({"paths": gopro_photo_paths})

len(df)

## 3. Extract date and coordinate from each available photo

In [ ]:
def extract_date_lat_lon(img_path):
    img_bytes = open(img_path, "rb")
    tags = exifread.process_file(img_bytes)

    required_keys = [
        "Image DateTime",
        "GPS GPSLatitude",
        "GPS GPSLongitude",
        "GPS GPSLatitudeRef",
        "GPS GPSLongitudeRef"
    ]
    if not all(key in tags for key in required_keys):
        return None, None, None

    # Extract date
    image_datetime = str(tags["Image DateTime"])

    # Convert to Python datetime object
    dt = datetime.strptime(image_datetime, "%Y:%m:%d %H:%M:%S")

    def convert_to_degrees(coord):
        """ Convert the GPS coordinates stored in the EXIF to degress in float format"""
        d = float(coord.values[0].num) / float(coord.values[0].den)
        m = float(coord.values[1].num) / float(coord.values[1].den)
        s = float(coord.values[2].num) / float(coord.values[2].den)
        return d + (m / 60.0) + (s / 3600.0)

    lat = convert_to_degrees(tags["GPS GPSLatitude"])
    lon = convert_to_degrees(tags["GPS GPSLongitude"])

    if tags["GPS GPSLatitudeRef"].values[0] != "N":
        lat = 0 - lat
    if tags["GPS GPSLongitudeRef"].values[0] != "E":
        lon = 0 - lon

    return dt, lat, lon

# Extract date and lat lon from each image
df[["date", "lat", "lon"]] = df["paths"].progress_apply(extract_date_lat_lon).to_list()

In [ ]:
print(f"Total photos: {len(df)}")
df = df[~df["lon"].isna()].copy()
print(f"Photos with valid exif tags: {len(df)}")

In [11]:
gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df["lon"], df["lat"])])

In [ ]:
from shapely.geometry import Point
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

# Obtain Admin Boundaries from GADM: https://gadm.org/data.html
gdf_gadm2 = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_KEN_2.json")

padding = 0.05
x_min, x_max = [df["lon"].min() - padding, df["lon"].max() + padding]
y_min, y_max = [df["lat"].min() - padding, df["lat"].max() + padding]

_, axes = plt.subplots(1, 2, figsize=(12, 10))
for ax in axes:
    gdf_gadm2.plot(ax=ax, facecolor="lightgray", edgecolor="black")
    ax.axis("off")

gdf.plot(ax=axes[0], color='red', markersize=1)
gdf.plot(ax=axes[1], color='red', markersize=3)
rect = Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, edgecolor='blue', facecolor='none')
axes[0].add_patch(rect)
axes[1].set_xlim([x_min, x_max]) # For zoomed in version
axes[1].set_ylim([y_min, y_max]) # For zoomed in version

for x, y, label in zip(gdf_gadm2.geometry.centroid.x, gdf_gadm2.geometry.centroid.y, gdf_gadm2['NAME_2']):
  if x_min <= x <= x_max and y_min <= y <= y_max:
    axes[1].text(x, y, label, fontsize=8, ha='center')

axes[1].set_title("Zoomed In", fontsize=15);

## 4. Classify as crop or not crop

In [ ]:
# Download CropNop model weights
!gsutil cp gs://street2sat-models/cropnop_v1.torchscript.pt .

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CropNop model
cropnop_model = torch.jit.load("/content/cropnop_v1.torchscript.pt", map_location=device)

def is_crop_or_not(img_path):

    # Preprocess image
    img = plt.imread(img_path)
    img = cv2.resize(img, (300, 300)) / 255
    img = img.transpose(2, 0, 1).astype("float32")
    img_tensor = torch.from_numpy(img).float().to(device)

    # Make crop or not prediction
    output = cropnop_model(img_tensor.unsqueeze(0))
    is_crop = (output <= 0).item()
    return is_crop

In [ ]:
# Display 9 example CropNop model predictions
preds = [is_crop_or_not(df["paths"].iloc[i]) for i in range(9)]
images = [plt.imread(df["paths"].iloc[i]) for i in range(9)]
fig, axes = plt.subplots(3, 3, figsize=(12, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title(f"Image {i}: {'Crop' if preds[i] else 'Not crop'}")
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 20 mins for 10k images
cropnop_run = input("Run CropNop model on all photos? [y/n]:") == "y"
if cropnop_run:
    print("Running CropNop model on all photos")
    df["is_crop"] = df["paths"].progress_apply(is_crop_or_not)
else:
    print("Skipping CropNop model run")
    df["is_crop"] = True

df["is_crop"].value_counts()

## 5. Segment crops

In [ ]:
# Download CropSeg model weights
!gsutil cp gs://street2sat-models/cropseg_v1.torchscript.pt .

In [19]:
from skimage.io import imread
from skimage.transform import resize

import numpy as np
import os

os.environ["LRU_CACHE_CAPACITY"] = "1"

# Load CropSeg model
cropseg_model = torch.jit.load("/content/cropseg_v1.torchscript.pt", map_location=device)

CLASSES = [
    "background",
    "banana",
    "maize",
    "rice",
    "soybean",
    "sugarcane",
    "sunflower",
    "tobacco",
    "wheat",
]

def segment_crops(img_path):
    img = imread(img_path)
    img = resize(img, (800, 800))
    img = img.astype(float)
    img = (
        255 * (img - np.min(img[:])) / (np.max(img[:]) - np.min(img[:]) + 0.1)
    ).astype(float)
    img = (img + 0.5) / 256
    gamma = -1 / np.nanmean(np.log(img))
    img = img ** (gamma)
    img_transposed = img.transpose(2, 0, 1).astype("float32")
    img_tensor = torch.from_numpy(img_transposed).unsqueeze(0).to(device)
    return img, cropseg_model(img_tensor)[0].cpu().detach().numpy()


def segment_crops_w_proportions(img_path):
    _, output = segment_crops(img_path)
    image_size = output.shape[1] * output.shape[2]
    segmentation_proportions = {
        crop:  round(output[i].sum() / image_size, 4) for i, crop in enumerate(CLASSES)
    }
    return segmentation_proportions

In [20]:
df_crops = df[df['is_crop']].copy()

In [ ]:
n_cols = 2
n_rows = 4
fig, axes = plt.subplots(n_rows, n_cols, figsize=(6, 14))
for i in range(4):
    img_path = df_crops["paths"].iloc[i]
    img, output = segment_crops(img_path)
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f"Image {i}")
    axes[i, 0].axis('off')

    segmented_img = output.argmax(axis=0)
    axes[i, 1].imshow(segmented_img, cmap='tab10', vmin=0, vmax=len(CLASSES))
    axes[i, 1].set_title(f"Segmentation {i}")
    axes[i, 1].axis('off')

    props = segment_crops_w_proportions(img_path)
    label = ""
    for crop, prop in sorted(props.items(), key=lambda item: item[1], reverse=True):
        if prop > 0.001:
            label += (f"{crop}: {round(prop, 4)}\n")

    axes[i, 1].text(0.05, 0.95, label, transform=axes[i, 1].transAxes, ha='left', va="top", color="white")

plt.tight_layout()
plt.show()

In [ ]:
# ~10 mins for 500 images, 1hr for 2k images
# TODO can probably make predictions faster through batches

segrun = input("Run segmentation model? [y/n]:").lower() == "y"
if segrun:
    print("Running segmentation model on all crop photos")
    df_crops["segmentation_proportions"] = df_crops["paths"].progress_apply(segment_crops_w_proportions)
    proportion_columns = pd.json_normalize(df_crops["segmentation_proportions"]).set_index(df_crops.index)
    df_crop_prop = pd.concat([df_crops, proportion_columns], axis=1)
    crops = list(proportion_columns.columns[1:])
    df_crop_prop["dominant_crop"] = df_crop_prop[crops].apply(lambda x: max(dict(x), key=dict(x).get), axis=1)
else:
    print("Skipping segmentation model run")
    df_crop_prop = df_crops
    df_crop_prop["dominant_crop"] = "_"

## 6. Filter out low confidence predictions

In [ ]:
if segrun:
    # Only points with less than 95% background kept
    bg_threshold = 0.99
    print(f"Before background filter: {len(df_crop_prop)}")
    df_crops_filtered = df_crop_prop[df_crop_prop["background"] < bg_threshold ].copy()
    print(f"After background filter: {len(df_crops_filtered)}")
else:
    df_crops_filtered = df_crop_prop.copy()


## 7. Move coordinate to field

In [ ]:
import utm
from datetime import timedelta
import math

# TODO add visual example

In [ ]:
# Copied and pasted from field_coord_distance_offset.ipynb

floor10 = lambda x: x//10 * 10
to_pixel_centroid = lambda coord: (floor10(coord[0]) + 5, floor10(coord[1]) + 5)

def generate_offset_point_wgs84(coord0, coord1, is_right_hand_drive=True, meters=20):
    utm_coord0 = utm.from_latlon(coord0[0], coord0[1])
    utm_coord1 = utm.from_latlon(coord1[0], coord1[1])

    for i, zone_type in [(2, "number"), (3, "letter")]:
        if utm_coord1[i] != utm_coord0[i]:
            print(utm_coord0)
            print(utm_coord1)
            raise ValueError(f"UTM Zone {zone_type} mismatch: {utm_coord0[i]} and {utm_coord1[i]}")


    delta_east = utm_coord1[0] - utm_coord0[0]
    delta_north = utm_coord1[1] - utm_coord0[1]

    # Offset for meters change in offset point distance
    x_offset = np.abs(meters * math.cos(math.atan(delta_east / delta_north)))

    # Direction of offset
    x_direction = np.sign(delta_north) if is_right_hand_drive else -np.sign(delta_north)
    x_offset *= x_direction

    orthogonal_slope = -delta_east / delta_north
    orthogonal_b = utm_coord1[1] - (orthogonal_slope * utm_coord1[0])
    orthogonal_y = lambda x: orthogonal_slope*x + orthogonal_b

    field_point_x = utm_coord1[0] + x_offset
    field_point_y = orthogonal_y(field_point_x)

    field_latlon = utm.to_latlon(field_point_x, field_point_y, utm_coord1[2], utm_coord1[3])

    pixel_centroid_x, pixel_centroid_y  = to_pixel_centroid((field_point_x, field_point_y))
    pixel_centroid_field_latlon = utm.to_latlon(pixel_centroid_x, pixel_centroid_y, utm_coord1[2], utm_coord1[3])

    return field_latlon, pixel_centroid_field_latlon, (delta_east, delta_north)

def road_pixel_centroid(coord):
    utm_coord = utm.from_latlon(coord[0], coord[1])
    utm_pixel_centroid = to_pixel_centroid(utm_coord)
    return utm.to_latlon(*utm_pixel_centroid, utm_coord[2], utm_coord[3])

In [ ]:
field_points = []
meters = 20

for i in tqdm(range(0, len(df_crops_filtered))):

    # Get road coordinate
    current_record = df_crops_filtered.iloc[i]
    road_coord = current_record["lat"], current_record["lon"]
    road_10m_centroid = road_pixel_centroid(road_coord)

    # Get prior coordinate
    time1 = current_record["date"]
    before_time_interval = time1 - timedelta(seconds=10)
    time_filter = (df["date"] < str(time1)) & (df["date"] > str(before_time_interval))
    prior_records = df[time_filter].sort_values(by=['date'])
    if len(prior_records) == 0:
        print(f"No prior records found for {i}")
        continue

    prior_record = prior_records.iloc[-1]
    prior_coord = prior_record["lat"], prior_record["lon"]

    # Get direction and field offset
    try:
        output = generate_offset_point_wgs84(prior_coord, road_coord, is_right_hand_drive, meters)
        offset_field_coord, offset_field_pixel_centroid, driving_direction = output

        field_points.append({
            "road_pixel_centroid": road_10m_centroid,
            "is_right_hand_drive": is_right_hand_drive,
            "driving_easting": driving_direction[0],
            "driving_northing": driving_direction[1],
            "offset_field_coord": offset_field_coord,
            "offset_field_pixel_centroid": offset_field_pixel_centroid,
            "time_computed": datetime.now(),
            **df_crops_filtered.iloc[i],
        })
    except Exception as e:
        print(f"Index: {i}, Exception: {e}")


In [ ]:
df_crop_type_w_duplicates = pd.DataFrame(field_points)
print(f"Points BEFORE deduplicating: {len(df_crop_type_w_duplicates)}")

In [ ]:
# Deduplication by selecting max dominant crop
def get_max_dominant_crop(group):
  max_idx = group.apply(lambda x: x[x['dominant_crop']], axis=1).idxmax()
  return group.loc[max_idx]

if segrun:
    # create new DataFrame with no duplicate - using groupby and applying get_max_dominant_crop on each group
    df_crop_type = df_crop_type_w_duplicates.groupby('offset_field_pixel_centroid').apply(get_max_dominant_crop).reset_index(drop=True)
else:
    df_crop_type = df_crop_type_w_duplicates.drop_duplicates(subset=['offset_field_pixel_centroid']).reset_index(drop=True)

print(f"Points AFTER deduplicating: {len(df_crop_type)}")

## 8. Get Admin Zones for each point

In [ ]:
geometry = [Point(xy) for xy in zip(df_crop_type["lon"], df_crop_type["lat"])]
gdf_points = gpd.GeoDataFrame(df_crop_type, geometry=geometry, crs="EPSG:4326")
gdf_points_gadm2 = gpd.sjoin(gdf_points, gdf_gadm2, how='left', predicate="within")

df_crop_type["GADM1"] = gdf_points_gadm2["NAME_1"]
df_crop_type["GADM2"] = gdf_points_gadm2["NAME_2"]

In [ ]:
df_crop_type["GADM1"].value_counts()

In [ ]:
df_crop_type["GADM2"].value_counts()

## 9. Create KMZ file(s)

In [ ]:
import simplekml

found_admin1_zones = df_crop_type[~df_crop_type["GADM1"].isna()]["GADM1"].unique()
DATA_GADM1_ZONES =  "_".join(found_admin1_zones)
if DATA_GADM1_ZONES not in PREFIX:
    PREFIX += f"_{DATA_GADM1_ZONES}"

DATA_YEARS = "_".join([str(year) for year in df_crop_type["date"].dt.year.unique()])
if DATA_YEARS not in PREFIX:
    PREFIX += f"_{DATA_YEARS}"

if segrun:
    DATA_BG_THRESHOLD = f"bg{int(bg_threshold*100)}"
    PREFIX += f"_{DATA_BG_THRESHOLD}"
else:
    PREFIX += f"_no_segrun"

# Change if necessary
PREFIX

In [ ]:
GCLOUD_PATH_INPUT

In [ ]:
def create_endpoint(image_path):
    endpoint_suffix = str(image_path).replace(str(image_folder), "")
    return GCLOUD_PATH_INPUT + endpoint_suffix

def create_description(record, image_path):
    image_name = Path(image_path).name
    endpoint = create_endpoint(image_path)
    a_href = ""
    if GCLOUD_PATH_INPUT != "":
        a_href = f"<a href='https://storage.cloud.google.com/{endpoint}'>https://storage.cloud.google.com/{endpoint}</a>"

    segrun_proportions = ""
    if segrun:
        segrun_proportions = f"""
<h2>CropSeg Model Prediction</h2>
<p>{record['segmentation_proportions']}</p>
"""

    return f"""
<img src='files/{image_name}' width='900px'/>
<br/>
<h2>{endpoint}</h2>
<p>Capture Time: {record['date']}</p>
{a_href}

<h2>Location</h2>
<p>GADM1: {record['GADM1']}</p>
<p>GADM2: {record['GADM2']}</p>
<p>Road Lat Lon: {record['lat']}, {record['lon']}</p>
<p>Field Lat Lon:  {record["offset_field_pixel_centroid"]}</p>


<h2>Driving Direction</h2>
<p>Northing: {record['driving_northing']}</p>
<p>Easting: {record['driving_easting']}</p>
<p>Is Right Hand Drive: {record['is_right_hand_drive']}</p>

{segrun_proportions}
"""
if GCLOUD_PATH_INPUT != "":
    print("Tester Link:")
    print(f"https://storage.cloud.google.com/{create_endpoint(df_crop_type['paths'].iloc[0])}")

In [ ]:
# Create KMZ file for every 100 points (more points make the KMZ laggy)
num_records = len(df_crop_type)

for range_start in range(0, num_records, 100):
    if range_start + 100 < num_records:
        range_end = range_start + 100
    else:
        range_end = num_records

    kml_document_name = PREFIX + f"_{range_start}_{range_end}"
    if "drive/MyDrive" not in kml_document_name:
        kml_document_name = f"drive/MyDrive/{kml_document_name}"

    kml = simplekml.Kml()
    kml.document.name = kml_document_name

    for _, record in tqdm(df_crop_type[range_start:range_end].iterrows()):
        latlon = record["offset_field_pixel_centroid"]
        image_path = record['paths']
        kml.newpoint(
            coords=[(latlon[1], latlon[0])],  # lon, lat optional height
            description=create_description(record, image_path),
            name=record["dominant_crop"],
            timestamp=record["date"]
        )
        kml.addfile(image_path)


    kml.savekmz(f"{kml_document_name}.kmz", format=False)

print("KMZ files saved to your Google Drive.")

The uploaded KMZ files can now be downloaded onto a computer with Google Earth Pro.

A Quality Assessment must be conducted following this protocol:

https://docs.google.com/document/d/1OCF2gpCQQbZP-y6xcTbKE2OzhkxMtyaJi8wiWi8jfzs/edit?usp=sharing